In [1]:
import pandas as pd
from mplsoccer import Sbopen, Pitch
import numpy as np
import math

In [2]:
parser = Sbopen()
df_match = parser.match(competition_id=37, season_id=42)
match_ids = df_match.match_id.unique()

In [4]:
all_events_df = []
for match_id in match_ids:
    events = parser.event(match_id)[0]
    # Filter for shot events that are free kicks
    free_kick_shots = events[(events['type_name'] == 'Shot') & (events['sub_type_name'] == 'Free Kick')]
    all_events_df.append(free_kick_shots)
df_shot = pd.concat(all_events_df, ignore_index=True)
df_shot.columns.tolist()

['id',
 'index',
 'period',
 'timestamp',
 'minute',
 'second',
 'possession',
 'duration',
 'match_id',
 'type_id',
 'type_name',
 'possession_team_id',
 'possession_team_name',
 'play_pattern_id',
 'play_pattern_name',
 'team_id',
 'team_name',
 'tactics_formation',
 'player_id',
 'player_name',
 'position_id',
 'position_name',
 'pass_recipient_id',
 'pass_recipient_name',
 'pass_length',
 'pass_angle',
 'pass_height_id',
 'pass_height_name',
 'end_x',
 'end_y',
 'sub_type_id',
 'sub_type_name',
 'body_part_id',
 'body_part_name',
 'x',
 'y',
 'under_pressure',
 'outcome_id',
 'outcome_name',
 'aerial_won',
 'counterpress',
 'out',
 'ball_recovery_recovery_failure',
 'pass_assisted_shot_id',
 'pass_shot_assist',
 'shot_statsbomb_xg',
 'end_z',
 'shot_key_pass_id',
 'technique_id',
 'technique_name',
 'goalkeeper_position_id',
 'goalkeeper_position_name',
 'pass_no_touch',
 'foul_committed_advantage',
 'foul_won_advantage',
 'pass_goal_assist',
 'foul_won_defensive',
 'pass_cut_back'

In [33]:
df_shot.columns.to_list()
df_goals = df_shot[df_shot.outcome_name == 'Goal'][['x', 'y', 'outcome_name', 'shot_statsbomb_xg']].copy()


df_non_goal_shots = df_shot[df_shot.outcome_name != 'Goal'][['x', 'y', 'outcome_name', 'shot_statsbomb_xg']].copy()
df_non_goal_shots['outcome_name'] = 'No Goal'

In [34]:
def calculate_angle(x, y):
  # 44 and 36 is the location of each goal post
  g0 = [120, 44]
  p = [x, y]
  g1 = [120, 36]

  v0 = np.array(g0) - np.array(p)
  v1 = np.array(g1) - np.array(p)

  angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
  return(abs(np.degrees(angle)))

def calculate_distance(x, y):
  x_dist = 120-x
  y_dist = 0
  if (y<36):
    y_dist = 36-y
  elif (y>44):
    y_dist = y-44
  return math.sqrt(x_dist**2 + y_dist**2)

In [35]:
df_goals['distance'] = df_goals.apply(lambda row: calculate_distance(row['x'], row['y']), axis=1)
df_goals['angle'] = df_goals.apply(lambda row: calculate_angle(row['x'], row['y']), axis=1)

# Add distance and angle columns to df_non_goal_shots
df_non_goal_shots['distance'] = df_non_goal_shots.apply(lambda row: calculate_distance(row['x'], row['y']), axis=1)
df_non_goal_shots['angle'] = df_non_goal_shots.apply(lambda row: calculate_angle(row['x'], row['y']), axis=1)

C:\Users\rrayn\AppData\Local\Temp\ipykernel_16708\2619458180.py:10: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
C:\Users\rrayn\AppData\Local\Temp\ipykernel_16708\2619458180.py:10: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))


In [38]:
df = pd.concat([df_goals, df_non_goal_shots], axis=0)

# Reset index (optional)
df = df.reset_index(drop=True)
df


,x,y,outcome_name,shot_statsbomb_xg,distance,angle
0,93.6,75.6,Goal,0.005921,41.176692,6.186767
1,101.2,40.0,Goal,0.121448,18.800000,24.022957
2,97.6,51.2,Goal,0.074687,23.528706,16.340806
3,90.2,53.7,Goal,0.015563,31.338953,12.678364
4,100.5,26.2,Goal,0.068447,21.824069,15.708010
...,...,...,...,...,...,...
65,83.7,38.0,No Goal,0.012272,36.300000,12.539122
66,94.1,26.1,No Goal,0.041974,27.727604,13.730289
67,86.0,35.3,No Goal,0.020629,34.007205,13.173558
68,108.5,15.3,No Goal,0.008374,23.679949,7.218783


In [39]:
summary_stats = df.groupby('outcome_name')[['distance', 'angle']].mean().round(2)
print(summary_stats)

              distance  angle
outcome_name                 
Goal             27.17  15.24
No Goal          28.61  13.54
